# Yeast Colony Growth Simulation: Normal vs Snowflake Yeast

This notebook simulates and compares two fundamentally different yeast growth paradigms:

1. **Normal Yeast** (*Saccharomyces cerevisiae* wild-type): Cells divide and separate, forming dispersed colonies that expand radially through the substrate.

2. **Snowflake Yeast** (*S. cerevisiae* ACE2 knockout): Cells divide but remain permanently attached via chitinous bud scars, creating fractal-like branched tree structures. This represents an experimental model for the evolution of multicellularity.

Both models are implemented as cellular automata with animated visualizations.

## Background

Snowflake yeast were created in the Ratcliff lab (Georgia Tech) by knocking out the ACE2 transcription factor, which normally activates genes responsible for degrading the cell wall connection between mother and daughter cells after budding. The resulting "multicellular" clusters undergo:
- Settling selection (larger clusters settle faster)
- Apoptotic division (branches break off to reproduce)
- Evolution of increased cell elongation (aspect ratio 1.2 → 2.7 over ~3000 generations)

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Ellipse, Circle
from matplotlib.collections import PatchCollection, EllipseCollection
from IPython.display import HTML, display
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Dict
import random
from collections import deque

# For reproducibility
np.random.seed(42)
random.seed(42)

# Animation settings
plt.rcParams['animation.embed_limit'] = 50  # MB limit for embedded animations

---

## Part 1: Normal Yeast Colony Simulation

Normal yeast colonies grow through a combination of:
- **Cell division**: Mother cells bud to produce daughter cells
- **Cell separation**: Daughters detach and become independent
- **Nutrient diffusion**: Growth rate depends on local nutrient availability
- **Crowding effects**: Cells compete for space

We model this as a cellular automaton on a 2D grid where each cell can:
- Divide into an adjacent empty space
- Die if nutrient-starved
- Remain quiescent if surrounded

In [ ]:
@dataclass
class NormalYeastParams:
    """Parameters for normal yeast colony simulation."""
    grid_size: int = 200               # Grid dimensions
    initial_cells: int = 5             # Starting colony size
    division_prob: float = 0.3         # Base division probability per step
    death_prob: float = 0.01           # Death probability when nutrient-starved
    nutrient_diffusion: float = 0.2    # Nutrient diffusion coefficient
    nutrient_consumption: float = 0.1  # Nutrient consumed per cell per step
    max_steps: int = 200               # Maximum simulation steps
    

class NormalYeastColony:
    """
    Cellular automaton simulation of normal (wild-type) yeast colony growth.
    
    The colony expands radially as cells divide and separate.
    Nutrient availability limits growth in the colony interior.
    """
    
    def __init__(self, params: NormalYeastParams = None):
        self.params = params or NormalYeastParams()
        self.grid_size = self.params.grid_size
        
        # Cell grid: 0 = empty, 1 = living cell
        self.cells = np.zeros((self.grid_size, self.grid_size), dtype=np.uint8)
        
        # Nutrient field: starts uniform, depleted by cells
        self.nutrients = np.ones((self.grid_size, self.grid_size), dtype=np.float32)
        
        # Cell age tracking (for visualization)
        self.cell_age = np.zeros((self.grid_size, self.grid_size), dtype=np.int32)
        
        # Initialize colony at center
        self._initialize_colony()
        
        # History for animation
        self.history = []
        self.step_count = 0
    
    def _initialize_colony(self):
        """Place initial cells at grid center."""
        center = self.grid_size // 2
        
        # Place initial cells in a small cluster
        for _ in range(self.params.initial_cells):
            offset_x = np.random.randint(-2, 3)
            offset_y = np.random.randint(-2, 3)
            x, y = center + offset_x, center + offset_y
            self.cells[x, y] = 1
            self.cell_age[x, y] = 1
    
    def _get_neighbors(self, x: int, y: int) -> List[Tuple[int, int]]:
        """Get valid 4-connected neighbor positions."""
        neighbors = []
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.grid_size and 0 <= ny < self.grid_size:
                neighbors.append((nx, ny))
        return neighbors
    
    def _diffuse_nutrients(self):
        """Diffuse nutrients using discrete Laplacian."""
        D = self.params.nutrient_diffusion
        
        # Laplacian stencil
        laplacian = (
            np.roll(self.nutrients, 1, axis=0) +
            np.roll(self.nutrients, -1, axis=0) +
            np.roll(self.nutrients, 1, axis=1) +
            np.roll(self.nutrients, -1, axis=1) -
            4 * self.nutrients
        )
        
        self.nutrients += D * laplacian
        
        # Boundary conditions: nutrients replenished at edges
        self.nutrients[0, :] = 1.0
        self.nutrients[-1, :] = 1.0
        self.nutrients[:, 0] = 1.0
        self.nutrients[:, -1] = 1.0
        
        # Cells consume nutrients
        self.nutrients -= self.params.nutrient_consumption * self.cells
        self.nutrients = np.clip(self.nutrients, 0, 1)
    
    def step(self) -> int:
        """
        Execute one simulation step.
        Returns number of new cells created.
        """
        # Diffuse and consume nutrients
        self._diffuse_nutrients()
        
        # Find all living cells
        living = np.argwhere(self.cells == 1)
        
        # Shuffle to randomize update order
        np.random.shuffle(living)
        
        new_cells = 0
        deaths = 0
        
        for x, y in living:
            # Age the cell
            self.cell_age[x, y] += 1
            
            local_nutrient = self.nutrients[x, y]
            
            # Death check (more likely when starved)
            death_prob = self.params.death_prob * (1 - local_nutrient)
            if np.random.random() < death_prob and self.cell_age[x, y] > 10:
                self.cells[x, y] = 0
                deaths += 1
                continue
            
            # Division check (more likely with nutrients)
            division_prob = self.params.division_prob * local_nutrient
            if np.random.random() < division_prob:
                # Find empty neighbor
                neighbors = self._get_neighbors(x, y)
                empty_neighbors = [(nx, ny) for nx, ny in neighbors 
                                   if self.cells[nx, ny] == 0]
                
                if empty_neighbors:
                    # Divide into random empty neighbor
                    nx, ny = random.choice(empty_neighbors)
                    self.cells[nx, ny] = 1
                    self.cell_age[nx, ny] = 1
                    new_cells += 1
        
        self.step_count += 1
        return new_cells
    
    def run(self, save_every: int = 5) -> List[np.ndarray]:
        """
        Run simulation for max_steps, saving snapshots.
        """
        self.history = [self.cells.copy()]
        
        for step in range(self.params.max_steps):
            self.step()
            
            if step % save_every == 0:
                self.history.append(self.cells.copy())
        
        return self.history
    
    def get_cell_count(self) -> int:
        """Return current number of living cells."""
        return int(np.sum(self.cells))
    
    def get_colony_radius(self) -> float:
        """Calculate effective colony radius from centroid."""
        living = np.argwhere(self.cells == 1)
        if len(living) == 0:
            return 0.0
        centroid = living.mean(axis=0)
        distances = np.linalg.norm(living - centroid, axis=1)
        return float(np.max(distances))

In [ ]:
# Run normal yeast simulation
print("Running normal yeast colony simulation...")
normal_params = NormalYeastParams(
    grid_size=150,
    initial_cells=3,
    division_prob=0.35,
    death_prob=0.005,
    max_steps=150
)

normal_colony = NormalYeastColony(normal_params)
normal_history = normal_colony.run(save_every=3)

print(f"Simulation complete: {len(normal_history)} frames")
print(f"Final cell count: {normal_colony.get_cell_count()}")
print(f"Colony radius: {normal_colony.get_colony_radius():.1f} grid units")

In [ ]:
def create_normal_yeast_animation(history: List[np.ndarray], 
                                   interval: int = 100) -> animation.FuncAnimation:
    """
    Create animated visualization of normal yeast colony growth.
    """
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Initial frame
    im = ax.imshow(history[0], cmap='YlOrBr', vmin=0, vmax=1,
                   interpolation='nearest')
    ax.set_title('Normal Yeast Colony Growth', fontsize=14)
    ax.set_xlabel('X position')
    ax.set_ylabel('Y position')
    
    # Text annotation for cell count
    count_text = ax.text(0.02, 0.98, '', transform=ax.transAxes,
                         fontsize=12, verticalalignment='top',
                         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    def update(frame):
        im.set_array(history[frame])
        cell_count = int(np.sum(history[frame]))
        count_text.set_text(f'Frame: {frame}\nCells: {cell_count}')
        return [im, count_text]
    
    anim = animation.FuncAnimation(fig, update, frames=len(history),
                                   interval=interval, blit=True)
    plt.close(fig)
    return anim

# Create and display animation
print("Creating normal yeast animation...")
normal_anim = create_normal_yeast_animation(normal_history, interval=80)
display(HTML(normal_anim.to_jshtml()))

---

## Part 2: Snowflake Yeast Simulation

Snowflake yeast (*S. cerevisiae* with ACE2 knockout) exhibit fundamentally different growth:

1. **Incomplete cytokinesis**: Daughter cells remain attached to mothers
2. **Branching tree topology**: The cluster forms a fractal-like tree structure
3. **No cell repositioning**: Once divided, cells cannot move relative to each other
4. **Cluster-level reproduction**: Large clusters break at weak points

We implement this as an L-system-inspired model where each cell tracks its parent connection, visualized as a 2D projection of the growing cluster.

In [ ]:
@dataclass
class SnowflakeYeastCell:
    """Represents a single cell in the snowflake yeast cluster."""
    position: np.ndarray           # 2D position
    radius: float                  # Cell radius
    orientation: float             # Angle in radians (long axis direction)
    aspect_ratio: float            # Length/width (1.0 = circular)
    generation: int                # Distance from founder
    age: int = 0                   # Division cycles since birth
    buds_produced: int = 0         # Number of daughters
    max_buds: int = 4              # Maximum daughter cells
    parent_id: Optional[int] = None
    cell_id: int = 0
    
    def can_divide(self) -> bool:
        return self.buds_produced < self.max_buds


@dataclass 
class SnowflakeYeastParams:
    """Parameters for snowflake yeast simulation."""
    # Cell properties
    initial_radius: float = 3.0        # Cell radius (arbitrary units)
    aspect_ratio: float = 1.2          # Ancestral aspect ratio
    evolved_aspect_ratio: float = 2.7  # Evolved (macroscopic) aspect ratio
    use_evolved: bool = False          # Use evolved morphology
    
    # Budding geometry
    bud_angle_mean: float = 45.0       # Mean polar angle from parent axis (degrees)
    bud_angle_std: float = 20.0        # Standard deviation
    bud_size_ratio: float = 0.8        # Daughter size relative to mother
    
    # Growth dynamics  
    division_prob: float = 0.6         # Per-cell division probability
    max_generations: int = 8           # Maximum tree depth
    max_cells: int = 300               # Maximum cluster size
    max_steps: int = 30                # Simulation duration
    
    # Collision
    collision_detection: bool = True
    min_separation: float = 0.9        # Fraction of sum of radii


class SnowflakeYeastCluster:
    """
    Simulation of snowflake yeast cluster growth.
    
    Unlike normal yeast, daughter cells remain permanently attached,
    forming a branching tree structure characteristic of primitive
    multicellularity.
    """
    
    def __init__(self, params: SnowflakeYeastParams = None):
        self.params = params or SnowflakeYeastParams()
        self.cells: List[SnowflakeYeastCell] = []
        self.next_id = 0
        self.step_count = 0
        self.history: List[List[SnowflakeYeastCell]] = []
        
        # Initialize with founder cell
        self._create_founder()
    
    def _create_founder(self):
        """Create the founding cell at origin."""
        aspect = (self.params.evolved_aspect_ratio if self.params.use_evolved 
                  else self.params.aspect_ratio)
        
        founder = SnowflakeYeastCell(
            position=np.array([0.0, 0.0]),
            radius=self.params.initial_radius,
            orientation=np.random.uniform(0, 2*np.pi),
            aspect_ratio=aspect,
            generation=0,
            cell_id=self.next_id
        )
        self.cells.append(founder)
        self.next_id += 1
    
    def _get_bud_position(self, mother: SnowflakeYeastCell) -> Tuple[np.ndarray, float]:
        """
        Calculate daughter position and orientation.
        
        Budding occurs at polar angle from mother's long axis,
        positioned at mother's surface.
        """
        # Random budding angle
        bud_angle = np.radians(
            self.params.bud_angle_mean + 
            np.random.normal(0, self.params.bud_angle_std)
        )
        
        # Random side (left or right of axis)
        side = np.random.choice([-1, 1])
        bud_angle *= side
        
        # Global angle = mother orientation + budding angle
        global_angle = mother.orientation + bud_angle
        
        # Distance from mother center to daughter center
        daughter_radius = mother.radius * self.params.bud_size_ratio
        separation = mother.radius + daughter_radius * 0.9  # Slight overlap at bud scar
        
        # Daughter position
        direction = np.array([np.cos(global_angle), np.sin(global_angle)])
        position = mother.position + direction * separation
        
        # Daughter orientation (slight deviation from budding direction)
        orientation = global_angle + np.random.normal(0, 0.2)
        
        return position, orientation
    
    def _check_collision(self, new_pos: np.ndarray, new_radius: float, 
                         exclude_id: Optional[int] = None) -> bool:
        """
        Check if a new cell would collide with existing cells.
        """
        if not self.params.collision_detection:
            return False
        
        for cell in self.cells:
            if cell.cell_id == exclude_id:
                continue
            
            dist = np.linalg.norm(new_pos - cell.position)
            min_dist = (cell.radius + new_radius) * self.params.min_separation
            
            if dist < min_dist:
                return True
        
        return False
    
    def step(self) -> int:
        """
        Execute one division cycle.
        Returns number of new cells created.
        """
        if len(self.cells) >= self.params.max_cells:
            return 0
        
        new_cells = []
        
        for cell in self.cells:
            cell.age += 1
            
            # Skip if cannot divide
            if not cell.can_divide():
                continue
            if cell.generation >= self.params.max_generations:
                continue
            if np.random.random() > self.params.division_prob:
                continue
            if len(self.cells) + len(new_cells) >= self.params.max_cells:
                break
            
            # Attempt division (try a few times if collision)
            for attempt in range(5):
                bud_pos, bud_orient = self._get_bud_position(cell)
                bud_radius = cell.radius * self.params.bud_size_ratio
                
                if not self._check_collision(bud_pos, bud_radius, cell.cell_id):
                    aspect = (self.params.evolved_aspect_ratio if self.params.use_evolved
                              else self.params.aspect_ratio)
                    
                    daughter = SnowflakeYeastCell(
                        position=bud_pos,
                        radius=bud_radius,
                        orientation=bud_orient,
                        aspect_ratio=aspect,
                        generation=cell.generation + 1,
                        parent_id=cell.cell_id,
                        cell_id=self.next_id
                    )
                    new_cells.append(daughter)
                    cell.buds_produced += 1
                    self.next_id += 1
                    break
        
        self.cells.extend(new_cells)
        self.step_count += 1
        return len(new_cells)
    
    def run(self) -> List[List[SnowflakeYeastCell]]:
        """
        Run simulation, saving snapshots at each step.
        """
        # Save initial state (deep copy)
        self.history = [self._copy_cells()]
        
        for _ in range(self.params.max_steps):
            added = self.step()
            self.history.append(self._copy_cells())
            
            if added == 0 and len(self.cells) > 5:
                # Add a few more frames at the end
                for _ in range(3):
                    self.history.append(self._copy_cells())
                break
        
        return self.history
    
    def _copy_cells(self) -> List[SnowflakeYeastCell]:
        """Create deep copy of current cell list."""
        return [
            SnowflakeYeastCell(
                position=c.position.copy(),
                radius=c.radius,
                orientation=c.orientation,
                aspect_ratio=c.aspect_ratio,
                generation=c.generation,
                age=c.age,
                buds_produced=c.buds_produced,
                max_buds=c.max_buds,
                parent_id=c.parent_id,
                cell_id=c.cell_id
            )
            for c in self.cells
        ]
    
    def get_connectivity(self) -> List[Tuple[int, int]]:
        """Get parent-child edges for tree visualization."""
        edges = []
        for cell in self.cells:
            if cell.parent_id is not None:
                edges.append((cell.parent_id, cell.cell_id))
        return edges

In [ ]:
# Run snowflake yeast simulation (ancestral morphology)
print("Running snowflake yeast simulation (ancestral)...")
snowflake_params = SnowflakeYeastParams(
    initial_radius=4.0,
    aspect_ratio=1.2,          # Ancestral: nearly circular
    use_evolved=False,
    division_prob=0.65,
    max_generations=7,
    max_cells=200,
    max_steps=25,
    bud_angle_mean=50.0,
    bud_angle_std=25.0
)

snowflake_cluster = SnowflakeYeastCluster(snowflake_params)
snowflake_history = snowflake_cluster.run()

print(f"Simulation complete: {len(snowflake_history)} frames")
print(f"Final cell count: {len(snowflake_cluster.cells)}")
print(f"Max generation: {max(c.generation for c in snowflake_cluster.cells)}")

In [ ]:
def create_snowflake_animation(history: List[List[SnowflakeYeastCell]], 
                                interval: int = 200,
                                show_connections: bool = True) -> animation.FuncAnimation:
    """
    Create animated visualization of snowflake yeast cluster growth.
    Cells are rendered as ellipses to show aspect ratio.
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Compute bounds from final frame
    final_cells = history[-1]
    if final_cells:
        all_x = [c.position[0] for c in final_cells]
        all_y = [c.position[1] for c in final_cells]
        max_r = max(c.radius for c in final_cells)
        margin = max_r * 3
        xlim = (min(all_x) - margin, max(all_x) + margin)
        ylim = (min(all_y) - margin, max(all_y) + margin)
        # Make square
        max_range = max(xlim[1] - xlim[0], ylim[1] - ylim[0]) / 2
        cx = (xlim[0] + xlim[1]) / 2
        cy = (ylim[0] + ylim[1]) / 2
        ax.set_xlim(cx - max_range, cx + max_range)
        ax.set_ylim(cy - max_range, cy + max_range)
    else:
        ax.set_xlim(-50, 50)
        ax.set_ylim(-50, 50)
    
    ax.set_aspect('equal')
    ax.set_title('Snowflake Yeast Cluster Growth', fontsize=14)
    ax.set_xlabel('X position (μm)')
    ax.set_ylabel('Y position (μm)')
    ax.set_facecolor('#f0f0f0')
    
    # Text annotation
    count_text = ax.text(0.02, 0.98, '', transform=ax.transAxes,
                         fontsize=12, verticalalignment='top',
                         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Colormap for generations
    cmap = plt.cm.viridis
    
    def update(frame):
        ax.clear()
        
        cells = history[frame]
        if not cells:
            return []
        
        # Restore axis settings
        if final_cells:
            ax.set_xlim(cx - max_range, cx + max_range)
            ax.set_ylim(cy - max_range, cy + max_range)
        ax.set_aspect('equal')
        ax.set_facecolor('#f0f0f0')
        ax.set_title('Snowflake Yeast Cluster Growth', fontsize=14)
        
        max_gen = max(c.generation for c in cells)
        
        # Draw connections first (behind cells)
        if show_connections:
            cell_dict = {c.cell_id: c for c in cells}
            for cell in cells:
                if cell.parent_id is not None and cell.parent_id in cell_dict:
                    parent = cell_dict[cell.parent_id]
                    ax.plot([parent.position[0], cell.position[0]],
                            [parent.position[1], cell.position[1]],
                            'k-', linewidth=0.5, alpha=0.3, zorder=1)
        
        # Draw cells as ellipses
        for cell in cells:
            color = cmap(cell.generation / max(max_gen, 1))
            
            # Ellipse dimensions
            width = cell.radius * 2 / cell.aspect_ratio
            height = cell.radius * 2
            angle = np.degrees(cell.orientation)
            
            ellipse = Ellipse(
                xy=cell.position,
                width=width,
                height=height,
                angle=angle,
                facecolor=color,
                edgecolor='black',
                linewidth=0.5,
                alpha=0.8,
                zorder=2
            )
            ax.add_patch(ellipse)
        
        # Update text
        count_text = ax.text(0.02, 0.98, f'Frame: {frame}\nCells: {len(cells)}',
                             transform=ax.transAxes, fontsize=12,
                             verticalalignment='top',
                             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        return []
    
    anim = animation.FuncAnimation(fig, update, frames=len(history),
                                   interval=interval, blit=False)
    plt.close(fig)
    return anim

# Create and display animation
print("Creating snowflake yeast animation...")
snowflake_anim = create_snowflake_animation(snowflake_history, interval=300)
display(HTML(snowflake_anim.to_jshtml()))

---

## Part 3: Evolved Macroscopic Snowflake Yeast

After ~3,000 generations of selection for larger size (settling selection), snowflake yeast evolved:

1. **Elongated cells**: Aspect ratio increased from ~1.2 to ~2.7
2. **Larger clusters**: ~20,000× more cells per cluster
3. **Increased toughness**: Material properties changed from gelatin-like to wood-like
4. **Branch entanglement**: Branches wrap around each other, increasing structural integrity

Let's simulate the evolved morphology:

In [ ]:
# Run evolved snowflake yeast simulation
print("Running evolved snowflake yeast simulation...")
evolved_params = SnowflakeYeastParams(
    initial_radius=4.0,
    aspect_ratio=2.7,          # Evolved: elongated cells
    evolved_aspect_ratio=2.7,
    use_evolved=True,
    division_prob=0.7,
    max_generations=9,
    max_cells=250,
    max_steps=30,
    bud_angle_mean=55.0,       # Slightly different budding angle
    bud_angle_std=20.0,
    collision_detection=True,
    min_separation=0.85
)

evolved_cluster = SnowflakeYeastCluster(evolved_params)
evolved_history = evolved_cluster.run()

print(f"Simulation complete: {len(evolved_history)} frames")
print(f"Final cell count: {len(evolved_cluster.cells)}")
print(f"Max generation: {max(c.generation for c in evolved_cluster.cells)}")

In [ ]:
# Create and display evolved animation
print("Creating evolved snowflake yeast animation...")
evolved_anim = create_snowflake_animation(evolved_history, interval=300)
display(HTML(evolved_anim.to_jshtml()))

---

## Part 4: Side-by-Side Comparison

Let's compare all three growth modes:

In [ ]:
def plot_comparison(normal_cells: np.ndarray,
                    ancestral_cells: List[SnowflakeYeastCell],
                    evolved_cells: List[SnowflakeYeastCell]):
    """
    Create side-by-side comparison of final colony morphologies.
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Panel 1: Normal yeast
    ax1 = axes[0]
    ax1.imshow(normal_cells, cmap='YlOrBr', interpolation='nearest')
    ax1.set_title(f'Normal Yeast\n({int(np.sum(normal_cells))} cells)', fontsize=12)
    ax1.set_xlabel('X position')
    ax1.set_ylabel('Y position')
    
    # Panel 2: Ancestral snowflake
    ax2 = axes[1]
    ax2.set_aspect('equal')
    ax2.set_facecolor('#f0f0f0')
    
    if ancestral_cells:
        all_x = [c.position[0] for c in ancestral_cells]
        all_y = [c.position[1] for c in ancestral_cells]
        max_r = max(c.radius for c in ancestral_cells)
        margin = max_r * 2
        max_range = max(max(all_x) - min(all_x), max(all_y) - min(all_y)) / 2 + margin
        cx, cy = np.mean(all_x), np.mean(all_y)
        ax2.set_xlim(cx - max_range, cx + max_range)
        ax2.set_ylim(cy - max_range, cy + max_range)
        
        max_gen = max(c.generation for c in ancestral_cells)
        cmap = plt.cm.viridis
        
        for cell in ancestral_cells:
            color = cmap(cell.generation / max(max_gen, 1))
            width = cell.radius * 2 / cell.aspect_ratio
            height = cell.radius * 2
            ellipse = Ellipse(
                xy=cell.position, width=width, height=height,
                angle=np.degrees(cell.orientation),
                facecolor=color, edgecolor='black', linewidth=0.3, alpha=0.8
            )
            ax2.add_patch(ellipse)
    
    ax2.set_title(f'Ancestral Snowflake Yeast\n({len(ancestral_cells)} cells, AR={ancestral_cells[0].aspect_ratio:.1f})', 
                  fontsize=12)
    
    # Panel 3: Evolved snowflake
    ax3 = axes[2]
    ax3.set_aspect('equal')
    ax3.set_facecolor('#f0f0f0')
    
    if evolved_cells:
        all_x = [c.position[0] for c in evolved_cells]
        all_y = [c.position[1] for c in evolved_cells]
        max_r = max(c.radius for c in evolved_cells)
        margin = max_r * 2
        max_range = max(max(all_x) - min(all_x), max(all_y) - min(all_y)) / 2 + margin
        cx, cy = np.mean(all_x), np.mean(all_y)
        ax3.set_xlim(cx - max_range, cx + max_range)
        ax3.set_ylim(cy - max_range, cy + max_range)
        
        max_gen = max(c.generation for c in evolved_cells)
        cmap = plt.cm.plasma
        
        for cell in evolved_cells:
            color = cmap(cell.generation / max(max_gen, 1))
            width = cell.radius * 2 / cell.aspect_ratio
            height = cell.radius * 2
            ellipse = Ellipse(
                xy=cell.position, width=width, height=height,
                angle=np.degrees(cell.orientation),
                facecolor=color, edgecolor='black', linewidth=0.3, alpha=0.8
            )
            ax3.add_patch(ellipse)
    
    ax3.set_title(f'Evolved Snowflake Yeast\n({len(evolved_cells)} cells, AR={evolved_cells[0].aspect_ratio:.1f})', 
                  fontsize=12)
    
    plt.tight_layout()
    return fig

# Generate comparison plot
fig = plot_comparison(
    normal_history[-1],
    snowflake_history[-1],
    evolved_history[-1]
)
plt.show()

---

## Part 5: Quantitative Analysis

In [ ]:
def analyze_cluster(cells: List[SnowflakeYeastCell], name: str) -> Dict:
    """
    Compute morphological metrics for snowflake yeast cluster.
    """
    if len(cells) < 2:
        return {'name': name, 'n_cells': len(cells)}
    
    positions = np.array([c.position for c in cells])
    
    # Centroid and distances
    centroid = positions.mean(axis=0)
    distances = np.linalg.norm(positions - centroid, axis=1)
    
    # Radius of gyration
    Rg = np.sqrt(np.mean(distances**2))
    
    # Fractal dimension estimate (mass-radius scaling)
    r_bins = np.linspace(distances.min() + 0.1, distances.max(), 15)
    N_within = [np.sum(distances <= r) for r in r_bins]
    
    valid = (np.array(N_within) > 0) & (r_bins > 0)
    if np.sum(valid) > 3:
        log_r = np.log(r_bins[valid])
        log_N = np.log(np.array(N_within)[valid])
        D_fractal = np.polyfit(log_r, log_N, 1)[0]
    else:
        D_fractal = np.nan
    
    # Generation statistics
    generations = [c.generation for c in cells]
    max_gen = max(generations)
    
    # Branching ratio
    parent_counts = {}
    for cell in cells:
        if cell.parent_id is not None:
            parent_counts[cell.parent_id] = parent_counts.get(cell.parent_id, 0) + 1
    avg_branching = np.mean(list(parent_counts.values())) if parent_counts else 1.0
    
    return {
        'name': name,
        'n_cells': len(cells),
        'cluster_radius': distances.max(),
        'radius_of_gyration': Rg,
        'fractal_dimension': D_fractal,
        'max_generation': max_gen,
        'avg_branching_ratio': avg_branching,
        'aspect_ratio': cells[0].aspect_ratio
    }

# Analyze both snowflake variants
ancestral_metrics = analyze_cluster(snowflake_history[-1], 'Ancestral')
evolved_metrics = analyze_cluster(evolved_history[-1], 'Evolved')

# Print comparison table
print("\n" + "="*60)
print("MORPHOLOGICAL COMPARISON: Ancestral vs Evolved Snowflake Yeast")
print("="*60)
print(f"{'Metric':<25} {'Ancestral':>15} {'Evolved':>15}")
print("-"*60)

for key in ['n_cells', 'cluster_radius', 'radius_of_gyration', 
            'fractal_dimension', 'max_generation', 'avg_branching_ratio', 'aspect_ratio']:
    if key in ancestral_metrics:
        val1 = ancestral_metrics[key]
        val2 = evolved_metrics[key]
        if isinstance(val1, float):
            print(f"{key:<25} {val1:>15.2f} {val2:>15.2f}")
        else:
            print(f"{key:<25} {val1:>15} {val2:>15}")
print("="*60)

In [ ]:
# Plot growth curves
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: Cell count over time
ax1 = axes[0]
normal_counts = [int(np.sum(frame)) for frame in normal_history]
ancestral_counts = [len(frame) for frame in snowflake_history]
evolved_counts = [len(frame) for frame in evolved_history]

ax1.plot(normal_counts, 'o-', label='Normal Yeast', color='#D2691E', markersize=4)
ax1.plot(ancestral_counts, 's-', label='Ancestral Snowflake', color='#2E8B57', markersize=4)
ax1.plot(evolved_counts, '^-', label='Evolved Snowflake', color='#8B008B', markersize=4)

ax1.set_xlabel('Simulation Frame')
ax1.set_ylabel('Cell Count')
ax1.set_title('Colony Growth Over Time')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Generation distribution for snowflake variants
ax2 = axes[1]

ancestral_gens = [c.generation for c in snowflake_history[-1]]
evolved_gens = [c.generation for c in evolved_history[-1]]

max_gen = max(max(ancestral_gens), max(evolved_gens))
bins = np.arange(0, max_gen + 2) - 0.5

ax2.hist(ancestral_gens, bins=bins, alpha=0.6, label='Ancestral', color='#2E8B57')
ax2.hist(evolved_gens, bins=bins, alpha=0.6, label='Evolved', color='#8B008B')

ax2.set_xlabel('Generation (distance from founder)')
ax2.set_ylabel('Number of Cells')
ax2.set_title('Cell Distribution by Generation')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Summary

This notebook demonstrated three distinct yeast growth paradigms:

### Normal Yeast (Wild-type)
- Cells divide and **separate**
- Colony expands as a **continuous front**
- Growth limited by nutrient diffusion to interior
- Produces **compact, circular colonies**

### Ancestral Snowflake Yeast (ACE2 knockout)
- Daughter cells **remain attached** via bud scars
- Forms **branching tree structure**
- Nearly circular cells (aspect ratio ~1.2)
- Represents first step toward multicellularity

### Evolved Macroscopic Snowflake Yeast
- ~3,000 generations of settling selection
- **Elongated cells** (aspect ratio ~2.7)
- More **open, dendritic structure**
- Branch entanglement provides mechanical strength

The transition from individual cells to attached clusters represents a major evolutionary transition—the origin of multicellularity. Snowflake yeast provide a powerful experimental model for studying this process in real time.

### References

- Ratcliff, W.C., et al. (2012). "Experimental evolution of multicellularity." *PNAS*, 109(5), 1595-1600.
- Ratcliff, W.C., et al. (2015). "Origins of multicellular evolvability in snowflake yeast." *Nature Communications*, 6, 6102.
- Bozdag, G.O., et al. (2023). "De novo evolution of macroscopic multicellularity." *Nature*, 617, 747-754.